In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Input
from flask import Flask, request, jsonify

# Load and preprocess dataset
file_path = r"C:/Users/ASUS/Downloads/attack_dataset.csv"
df = pd.read_csv(file_path)

# Encode the labels (normal = 0, DoS = 1)
label_encoder = LabelEncoder()
df['label'] = label_encoder.fit_transform(df['label'])

# Split IP address into individual octets for easier model input
df[['octet1', 'octet2', 'octet3', 'octet4']] = df['ip_address'].str.split('.', expand=True).astype(int)
X = df[['octet1', 'octet2', 'octet3', 'octet4']].values
y = df['label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a neural network model
model = Sequential()
model.add(Input(shape=(4,)))  # Updated input specification
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=4)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

# Save the model
model.save('dos_detection_model.keras')  # Using the newer format

# Flask API to simulate WAF behavior
app = Flask(__name__)

# Load trained model
loaded_model = tf.keras.models.load_model('dos_detection_model.keras')

def predict_dos(ip_address):
    octets = list(map(int, ip_address.split('.')))
    input_data = np.array([octets])
    prediction = loaded_model.predict(input_data)
    return 'DoS' if prediction[0][0] >= 0.5 else 'normal'

@app.route('/check_ip', methods=['POST'])
def check_ip():
    data = request.json
    ip_address = data['ip_address']
    result = predict_dos(ip_address)
    
    # Simulate WAF action
    if result == 'DoS':
        return jsonify({"ip_address": ip_address, "status": "blocked"}), 403
    else:
        return jsonify({"ip_address": ip_address, "status": "allowed"}), 200

#if __name__ == "__main__":
#    app.run(debug=False)  # Turn off debug mode to avoid restarting issues

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000, debug=False)  

Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 1s 953us/step - accuracy: 0.0510 - loss: 10.6273     
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 691us/step - accuracy: 0.4917 - loss: 1.9196
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.9569 - loss: 0.6359
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.9176 - loss: 0.6358
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 643us/step - accuracy: 0.9573 - loss: 0.3541
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step - accuracy: 0.8973 - loss: 0.8547
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.9509 - loss: 0.6204
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.9763 - loss: 0.3064
Epoch 9/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.9544 - loss: 0.2913
Epoch 10/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 667us/step - accuracy: 0.9291 - loss: 0.2746
Epoch 11/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 619us/step - accuracy: 0.9634 - loss: 0.1845
Epoch 12/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 619u

 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.17.18.14:5000
Press CTRL+C to quit
172.17.18.14 - - [08/Nov/2024 23:08:59] "GET / HTTP/1.1" 404 -
172.17.18.14 - - [08/Nov/2024 23:09:00] "GET /favicon.ico HTTP/1.1" 404 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step


127.0.0.1 - - [08/Nov/2024 23:09:10] "POST /check_ip HTTP/1.1" 403 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step


127.0.0.1 - - [08/Nov/2024 23:09:11] "POST /check_ip HTTP/1.1" 200 -


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


127.0.0.1 - - [08/Nov/2024 23:09:11] "POST /check_ip HTTP/1.1" 200 -


In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text
